In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import keras_tuner as kt


### Define functions for use during optimization

In [2]:
def reduce_count_vals(df, colname, threshold):
    # Determine which values to replace if counts are less than ...?
    counts = df[colname].value_counts()
    replace_list = list(counts[counts < threshold].index)

    # Replace in dataframe
    for item in replace_list:
       df[colname] = df[colname].replace(item,"Other")
    
    # Check to make sure binning was successful
    df[colname].value_counts()

In [3]:
def do_one_hot(df, y_col):
    # Create a list of columns that are 'object' type
    obj_cat = df.dtypes[df.dtypes == "object"].index.tolist()
    # Create a OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)
    # Fit and transform the OneHotEncoder using the categorical variable list
    encode_df = pd.DataFrame(enc.fit_transform(df[obj_cat]))
    # Add the encoded variable names to the DataFrame
    encode_df.columns = enc.get_feature_names(obj_cat)
    
    # Merge one-hot encoded features and drop the originals
    df = df.merge(encode_df, left_index=True, right_index=True)
    df = df.drop(obj_cat,1)
    
    # Split our preprocessed data into our features and target arrays
    y = df[y_col].values
    df.drop(columns=[y_col], inplace=True)
    X = df.values
    print(f"merged df.shape()={df.shape}")
    
    return df, X, y

In [4]:
def do_scatter_plots(df):
    for xx in ['APPLICATION_TYPE', 
        'AFFILIATION', 
        'CLASSIFICATION', 
        'USE_CASE', 
        'ORGANIZATION', 
        'INCOME_AMT', 
              ]:
        for yy in ['APPLICATION_TYPE', 
            'AFFILIATION', 
            'CLASSIFICATION', 
            'USE_CASE', 
            'ORGANIZATION', 
            'INCOME_AMT'
                  ]:
            if xx == yy:
                continue
            titlestring = f"x={xx} vs. y={yy}"
            df.plot.scatter(x=xx, y=yy, title=titlestring, c='IS_SUCCESSFUL', colormap='winter')


In [5]:
def do_scale(scaler):
    # Scale the data
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    return X_train_scaled, X_test_scaled

In [6]:
def build_model(inputs, layers):
    nn = tf.keras.models.Sequential()
    first = True
    for layer in layers:
        if first:
            first = False
            nn.add(tf.keras.layers.Dense(units=layer['units'], activation=layer['act'], input_dim=inputs))
        else:
            nn.add(tf.keras.layers.Dense(units=layer['units'], activation=layer['act']))

    print(nn.summary())
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return nn

In [7]:
def train_nn_model(model, X_train, y_train, n_epochs, checkpoint_dir):
    # Create a callback that saves the model's weights every epoch
    from tensorflow.keras.callbacks import ModelCheckpoint
    
    # Define the checkpoint path and filenames
    os.makedirs("checkpoints_opt",exist_ok=True)
    #checkpoint_file = f"weights.{epoch:02d}.hdf5"
    #checkpoint_path = f"checkpoints_opt/weights.{epoch:02d}.hdf5"
    checkpoint_path = "checkpoints_opt/weights_2.{epoch:02d}.hdf5"
    
    cp_callback_opt = ModelCheckpoint(
        filepath=checkpoint_path,
        verbose=1,
        save_weights_only=True,
        period=5)

    # Normally we use 'save_freq', but it behaves strangely, and I could not get
    # it to save every 5 epochs. The 'period' param is now deprecated, but it works.
        #save_freq='epoch')
        
    fit_model = model.fit(X_train, y_train, epochs=n_epochs, callbacks=[cp_callback_opt])
    return fit_model

### Optimation Run 1 - With 7 layers of tanh & sigmoid

In [8]:
application_df = pd.read_csv("Resources/charity_data.csv")
csv_cols = [
    'EIN', 
    'NAME', 
    'APPLICATION_TYPE', 
    'AFFILIATION', 
    'CLASSIFICATION', 
    'USE_CASE', 
    'ORGANIZATION', 
    'STATUS', 
    'INCOME_AMT', 
    'SPECIAL_CONSIDERATIONS', 
    'ASK_AMT', 
    'IS_SUCCESSFUL'
]
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

reduce_count_vals(application_df, 'APPLICATION_TYPE', 500)
reduce_count_vals(application_df, 'CLASSIFICATION', 1500)

application_df, X, y = do_one_hot(application_df, 'IS_SUCCESSFUL')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_train_scaled, X_test_scaled = do_scale(StandardScaler())
#X_train_scaled, X_test_scaled = do_scale(MinMaxScaler())

nins = application_df.shape[1]
print(f"nins={nins}")
nn_model = build_model(inputs=nins, layers=[
    {'units': 340, 'act': 'tanh'},
    {'units': 170, 'act': 'tanh'},
    {'units': 85,  'act': 'tanh'},
    {'units': 40,  'act': 'tanh'},
    {'units': 20,  'act': 'sigmoid'},
    {'units': 5,   'act': 'sigmoid'},
    {'units': 1,   'act': 'sigmoid'}
])
trained_model = train_nn_model(nn_model, X_train_scaled, y_train, n_epochs=500, checkpoint_dir="checkpoints_opt")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_opt1.h5")


merged df.shape()=(34299, 43)
nins=43
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 340)               14960     
_________________________________________________________________
dense_1 (Dense)              (None, 170)               57970     
_________________________________________________________________
dense_2 (Dense)              (None, 85)                14535     
_________________________________________________________________
dense_3 (Dense)              (None, 40)                3440      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 105       
_________________________________________________________________
dense_6 (Dense)   

C:\Users\bjile\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  del sys.path[0]


Epoch 1/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5924 - accuracy: 0.7132
Epoch 2/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5672 - accuracy: 0.7293
Epoch 3/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5603 - accuracy: 0.7294
Epoch 4/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5567 - accuracy: 0.7306
Epoch 5/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7313

Epoch 00005: saving model to checkpoints_opt\weights_2.05.hdf5
Epoch 6/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5531 - accuracy: 0.7310
Epoch 7/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5505 - accuracy: 0.7332
Epoch 8/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5511 - accuracy: 0.7323
Epoch 9/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accuracy: 0.7320
Epoch 10/500
804/804 [=====

804/804 [==============================] - 2s 2ms/step - loss: 0.5349 - accuracy: 0.7414
Epoch 73/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5357 - accuracy: 0.7407
Epoch 74/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5353 - accuracy: 0.7401
Epoch 75/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5348 - accuracy: 0.7410

Epoch 00075: saving model to checkpoints_opt\weights_2.75.hdf5
Epoch 76/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5341 - accuracy: 0.7410
Epoch 77/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5347 - accuracy: 0.7411
Epoch 78/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5348 - accuracy: 0.7405
Epoch 79/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5354 - accuracy: 0.7410
Epoch 80/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5347 - accuracy: 0.7411

Epoch 00080: saving model to c

804/804 [==============================] - 2s 2ms/step - loss: 0.5336 - accuracy: 0.7420
Epoch 144/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5331 - accuracy: 0.7419
Epoch 145/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5332 - accuracy: 0.7423

Epoch 00145: saving model to checkpoints_opt\weights_2.145.hdf5
Epoch 146/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5334 - accuracy: 0.7417
Epoch 147/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 0.7422
Epoch 148/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 0.7419
Epoch 149/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5325 - accuracy: 0.7426
Epoch 150/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5326 - accuracy: 0.7416

Epoch 00150: saving model to checkpoints_opt\weights_2.150.hdf5
Epoch 151/500
804/804 [==============================] - 2s 

804/804 [==============================] - 2s 2ms/step - loss: 0.5325 - accuracy: 0.7418
Epoch 214/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5334 - accuracy: 0.7420
Epoch 215/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5338 - accuracy: 0.7417

Epoch 00215: saving model to checkpoints_opt\weights_2.215.hdf5
Epoch 216/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5323 - accuracy: 0.7425
Epoch 217/500
804/804 [==============================] - 1s 2ms/step - loss: 0.5321 - accuracy: 0.7422
Epoch 218/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5322 - accuracy: 0.7417
Epoch 219/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5323 - accuracy: 0.7431
Epoch 220/500
804/804 [==============================] - 1s 2ms/step - loss: 0.5339 - accuracy: 0.7409

Epoch 00220: saving model to checkpoints_opt\weights_2.220.hdf5
Epoch 221/500
804/804 [==============================] - 1s 

804/804 [==============================] - 2s 2ms/step - loss: 0.5327 - accuracy: 0.7416
Epoch 284/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5321 - accuracy: 0.7411
Epoch 285/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5325 - accuracy: 0.7409

Epoch 00285: saving model to checkpoints_opt\weights_2.285.hdf5
Epoch 286/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5318 - accuracy: 0.7414
Epoch 287/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5320 - accuracy: 0.7426
Epoch 288/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5325 - accuracy: 0.7413
Epoch 289/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5324 - accuracy: 0.7414
Epoch 290/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5321 - accuracy: 0.7413

Epoch 00290: saving model to checkpoints_opt\weights_2.290.hdf5
Epoch 291/500
804/804 [==============================] - 2s 

804/804 [==============================] - 2s 2ms/step - loss: 0.5322 - accuracy: 0.7420
Epoch 354/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 0.7423
Epoch 355/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5326 - accuracy: 0.7416

Epoch 00355: saving model to checkpoints_opt\weights_2.355.hdf5
Epoch 356/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5326 - accuracy: 0.7423
Epoch 357/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5325 - accuracy: 0.7422
Epoch 358/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5323 - accuracy: 0.7423
Epoch 359/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5329 - accuracy: 0.7420
Epoch 360/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5323 - accuracy: 0.7415

Epoch 00360: saving model to checkpoints_opt\weights_2.360.hdf5
Epoch 361/500
804/804 [==============================] - 2s 

804/804 [==============================] - 2s 3ms/step - loss: 0.5328 - accuracy: 0.7424
Epoch 424/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5328 - accuracy: 0.7421
Epoch 425/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5323 - accuracy: 0.7425

Epoch 00425: saving model to checkpoints_opt\weights_2.425.hdf5
Epoch 426/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5329 - accuracy: 0.7416
Epoch 427/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5328 - accuracy: 0.7420
Epoch 428/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5329 - accuracy: 0.7423
Epoch 429/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5334 - accuracy: 0.7421
Epoch 430/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5330 - accuracy: 0.7418

Epoch 00430: saving model to checkpoints_opt\weights_2.430.hdf5
Epoch 431/500
804/804 [==============================] - 2s 

804/804 [==============================] - 2s 3ms/step - loss: 0.5325 - accuracy: 0.7419
Epoch 494/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5329 - accuracy: 0.7420
Epoch 495/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5326 - accuracy: 0.7422

Epoch 00495: saving model to checkpoints_opt\weights_2.495.hdf5
Epoch 496/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5326 - accuracy: 0.7417
Epoch 497/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5326 - accuracy: 0.7422
Epoch 498/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5326 - accuracy: 0.7423
Epoch 499/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5325 - accuracy: 0.7419
Epoch 500/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5321 - accuracy: 0.7421

Epoch 00500: saving model to checkpoints_opt\weights_2.500.hdf5
268/268 - 0s - loss: 0.5622 - accuracy: 0.7269
Loss: 0.56222

### Optimization Run 2 - Same as Run1, but drop 'STATUS' & 'SPECIAL_CONSIDERATIONS'; and bucketize 'AFFILIATION', 'ORGANIZATION', AND 'INCOME_AMT'

In [9]:
application_df = pd.read_csv("Resources/charity_data.csv")
csv_cols = [
    'EIN', 
    'NAME', 
    'APPLICATION_TYPE', 
    'AFFILIATION', 
    'CLASSIFICATION', 
    'USE_CASE', 
    'ORGANIZATION', 
    'STATUS', 
    'INCOME_AMT', 
    'SPECIAL_CONSIDERATIONS', 
    'ASK_AMT', 
    'IS_SUCCESSFUL'
]

application_df.drop(columns=['EIN', 'NAME','STATUS', 'SPECIAL_CONSIDERATIONS'], inplace=True)

reduce_count_vals(application_df, 'APPLICATION_TYPE', 500)
reduce_count_vals(application_df, 'CLASSIFICATION', 1500)
reduce_count_vals(application_df, 'AFFILIATION', 100)
reduce_count_vals(application_df, 'ORGANIZATION', 500)
reduce_count_vals(application_df, 'INCOME_AMT', 500)

application_df, X, y = do_one_hot(application_df, 'IS_SUCCESSFUL')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
#X_train_scaled, X_test_scaled = do_scale(StandardScaler())
X_train_scaled, X_test_scaled = do_scale(MinMaxScaler())

nins = application_df.shape[1]
print(f"nins={nins}")
nn_model = build_model(inputs=nins, layers=[
    {'units': 340, 'act': 'relu'},
    {'units': 170, 'act': 'relu'},
    {'units': 85,  'act': 'relu'},
    {'units': 40,  'act': 'relu'},
    {'units': 20,  'act': 'sigmoid'},
    {'units': 5,   'act': 'sigmoid'},
    {'units': 1,   'act': 'sigmoid'}
])
trained_model = train_nn_model(nn_model, X_train_scaled, y_train, n_epochs=500, checkpoint_dir="checkpoints_opt")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_opt2.h5")

merged df.shape()=(34299, 34)
nins=34
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 340)               11900     
_________________________________________________________________
dense_8 (Dense)              (None, 170)               57970     
_________________________________________________________________
dense_9 (Dense)              (None, 85)                14535     
_________________________________________________________________
dense_10 (Dense)             (None, 40)                3440      
_________________________________________________________________
dense_11 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_12 (Dense)             (None, 5)                 105       
_________________________________________________________________
dense_13 (Dense)

C:\Users\bjile\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  del sys.path[0]


Epoch 1/500
804/804 [==============================] - 2s 2ms/step - loss: 0.6053 - accuracy: 0.6947
Epoch 2/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5671 - accuracy: 0.7296
Epoch 3/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5585 - accuracy: 0.7315
Epoch 4/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5548 - accuracy: 0.7343
Epoch 5/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5533 - accuracy: 0.7351

Epoch 00005: saving model to checkpoints_opt\weights_2.05.hdf5
Epoch 6/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7355
Epoch 7/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7343
Epoch 8/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7346
Epoch 9/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7356
Epoch 10/500
804/804 [=====

804/804 [==============================] - 2s 3ms/step - loss: 0.5354 - accuracy: 0.7409
Epoch 73/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5352 - accuracy: 0.7410
Epoch 74/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5354 - accuracy: 0.7412
Epoch 75/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5354 - accuracy: 0.7406

Epoch 00075: saving model to checkpoints_opt\weights_2.75.hdf5
Epoch 76/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5353 - accuracy: 0.7409
Epoch 77/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5355 - accuracy: 0.7413
Epoch 78/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5352 - accuracy: 0.7407
Epoch 79/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5354 - accuracy: 0.7412
Epoch 80/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5356 - accuracy: 0.7413

Epoch 00080: saving model to c

804/804 [==============================] - 2s 3ms/step - loss: 0.5345 - accuracy: 0.7410
Epoch 144/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5341 - accuracy: 0.7416
Epoch 145/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5343 - accuracy: 0.7418

Epoch 00145: saving model to checkpoints_opt\weights_2.145.hdf5
Epoch 146/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5342 - accuracy: 0.7416
Epoch 147/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5352 - accuracy: 0.7401
Epoch 148/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5347 - accuracy: 0.7409
Epoch 149/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5351 - accuracy: 0.7415
Epoch 150/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5349 - accuracy: 0.7414

Epoch 00150: saving model to checkpoints_opt\weights_2.150.hdf5
Epoch 151/500
804/804 [==============================] - 2s 

804/804 [==============================] - 2s 3ms/step - loss: 0.5349 - accuracy: 0.7414
Epoch 214/500
804/804 [==============================] - ETA: 0s - loss: 0.5352 - accuracy: 0.74 - 2s 3ms/step - loss: 0.5350 - accuracy: 0.7418
Epoch 215/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5352 - accuracy: 0.7414

Epoch 00215: saving model to checkpoints_opt\weights_2.215.hdf5
Epoch 216/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5380 - accuracy: 0.7416
Epoch 217/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5373 - accuracy: 0.7413
Epoch 218/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5361 - accuracy: 0.7418
Epoch 219/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5348 - accuracy: 0.7417
Epoch 220/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5350 - accuracy: 0.7413

Epoch 00220: saving model to checkpoints_opt\weights_2.220.hdf5
Epoch 221/500
804/

804/804 [==============================] - 2s 2ms/step - loss: 0.5357 - accuracy: 0.7409
Epoch 284/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5364 - accuracy: 0.7408
Epoch 285/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5354 - accuracy: 0.7415: 0s - loss: 0.5

Epoch 00285: saving model to checkpoints_opt\weights_2.285.hdf5
Epoch 286/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5362 - accuracy: 0.7408
Epoch 287/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5365 - accuracy: 0.7410
Epoch 288/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5369 - accuracy: 0.7414
Epoch 289/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5360 - accuracy: 0.7416
Epoch 290/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5364 - accuracy: 0.7417

Epoch 00290: saving model to checkpoints_opt\weights_2.290.hdf5
Epoch 291/500
804/804 [=====================

804/804 [==============================] - 2s 2ms/step - loss: 0.5403 - accuracy: 0.7410
Epoch 354/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5390 - accuracy: 0.7413
Epoch 355/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5384 - accuracy: 0.7420

Epoch 00355: saving model to checkpoints_opt\weights_2.355.hdf5
Epoch 356/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5391 - accuracy: 0.7406
Epoch 357/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5397 - accuracy: 0.7408
Epoch 358/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5419 - accuracy: 0.7402
Epoch 359/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5455 - accuracy: 0.7402
Epoch 360/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5415 - accuracy: 0.7408

Epoch 00360: saving model to checkpoints_opt\weights_2.360.hdf5
Epoch 361/500
804/804 [==============================] - 2s 

804/804 [==============================] - 2s 2ms/step - loss: 0.5437 - accuracy: 0.7400
Epoch 424/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5435 - accuracy: 0.7388
Epoch 425/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5423 - accuracy: 0.7409

Epoch 00425: saving model to checkpoints_opt\weights_2.425.hdf5
Epoch 426/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5422 - accuracy: 0.7409
Epoch 427/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5426 - accuracy: 0.7412
Epoch 428/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5450 - accuracy: 0.7393
Epoch 429/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5442 - accuracy: 0.7395
Epoch 430/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5460 - accuracy: 0.7397

Epoch 00430: saving model to checkpoints_opt\weights_2.430.hdf5
Epoch 431/500
804/804 [==============================] - 2s 

804/804 [==============================] - 2s 2ms/step - loss: 0.5938 - accuracy: 0.6946
Epoch 494/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5755 - accuracy: 0.7159
Epoch 495/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5725 - accuracy: 0.7191

Epoch 00495: saving model to checkpoints_opt\weights_2.495.hdf5
Epoch 496/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5745 - accuracy: 0.7219
Epoch 497/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5708 - accuracy: 0.7276
Epoch 498/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5716 - accuracy: 0.7263
Epoch 499/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5958 - accuracy: 0.7022
Epoch 500/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5847 - accuracy: 0.7164

Epoch 00500: saving model to checkpoints_opt\weights_2.500.hdf5
268/268 - 0s - loss: 0.5917 - accuracy: 0.7073
Loss: 0.59167

### Optimization Run 3 - with KerasTuner

In [10]:
# Create a method that creates a new Sequential model with hyperparameter options

def create_tuner_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=70,
        max_value=140,
        step=10), activation=activation, input_dim=40))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for ii in range(hp.Int('num_layers', 1, 10)):
        #nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(ii),
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(ii),                                             
            min_value=70,
            max_value=140,
            step=10),
            activation=activation))
            
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [11]:
def run_keras_tuner(X_train, y_train, n_epochs, validation_data):
    # Create a `Hyperband()` tuner instance
    tuner = kt.Hyperband(
        create_tuner_model,
        objective="val_accuracy",
        max_epochs=50,
        hyperband_iterations=2,
        overwrite=True)

    # Run the kerastuner search for best hyperparameters
    tuner.search(X_train, y_train, epochs=n_epochs, validation_data=validation_data)
    return tuner

In [12]:
application_df = pd.read_csv("Resources/charity_data.csv")
csv_cols = [
    'EIN', 
    'NAME', 
    'APPLICATION_TYPE', 
    'AFFILIATION', 
    'CLASSIFICATION', 
    'USE_CASE', 
    'ORGANIZATION', 
    'STATUS', 
    'INCOME_AMT', 
    'SPECIAL_CONSIDERATIONS', 
    'ASK_AMT', 
    'IS_SUCCESSFUL'
]

application_df.drop(columns=['EIN', 'NAME','STATUS', 'SPECIAL_CONSIDERATIONS'], inplace=True)

reduce_count_vals(application_df, 'APPLICATION_TYPE', 500)
reduce_count_vals(application_df, 'CLASSIFICATION', 1500)

application_df, X, y = do_one_hot(application_df, 'IS_SUCCESSFUL')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_train_scaled, X_test_scaled = do_scale(StandardScaler())
#X_train_scaled, X_test_scaled = do_scale(MinMaxScaler())

nins = application_df.shape[1]
print(f"nins={nins}")

tuner = run_keras_tuner(X_train_scaled, y_train, n_epochs=20, validation_data=(X_test_scaled,y_test))

# # Evaluate the model using the test data
# model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_opt3.h5")

Trial 180 Complete [00h 01m 37s]
val_accuracy: 0.727580189704895

Best val_accuracy So Far: 0.7304956316947937
Total elapsed time: 01h 11m 58s
INFO:tensorflow:Oracle triggered exit


In [13]:
# Get top 3 model hyperparameters and print the values
best_hypers = []
for ii in range(1,4):
    best_hyper = tuner.get_best_hyperparameters(ii)[0]
    print(best_hyper.values)
    best_hypers.append(best_hyper)

{'activation': 'relu', 'first_units': 110, 'num_layers': 5, 'units_0': 140, 'units_1': 80, 'units_2': 130, 'units_3': 70, 'units_4': 100, 'units_5': 110, 'units_6': 110, 'units_7': 110, 'units_8': 120, 'units_9': 90, 'tuner/epochs': 17, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 110, 'num_layers': 5, 'units_0': 140, 'units_1': 80, 'units_2': 130, 'units_3': 70, 'units_4': 100, 'units_5': 110, 'units_6': 110, 'units_7': 110, 'units_8': 120, 'units_9': 90, 'tuner/epochs': 17, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 110, 'num_layers': 5, 'units_0': 140, 'units_1': 80, 'units_2': 130, 'units_3': 70, 'units_4': 100, 'units_5': 110, 'units_6': 110, 'units_7': 110, 'units_8': 120, 'units_9': 90, 'tuner/epochs': 17, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}


In [14]:
# Evaluate the top 3 models against the test dataset
best_models = []
for ii in range(1,4):
    best_model = tuner.get_best_models(ii)[0]
    model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    best_models.append(best_model)

268/268 - 0s - loss: 0.5488 - accuracy: 0.7305
Loss: 0.5487930178642273, Accuracy: 0.7304956316947937
268/268 - 0s - loss: 0.5488 - accuracy: 0.7305
Loss: 0.5487930178642273, Accuracy: 0.7304956316947937
268/268 - 0s - loss: 0.5488 - accuracy: 0.7305
Loss: 0.5487930178642273, Accuracy: 0.7304956316947937


In [15]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))